In [ ]:
#Install Java Runtime 8
import os       #importing os to set environment variable
!apt-get remove -y java*
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'java-propose-classpath' for glob 'java*'
Note, selecting 'java2-sdk' for glob 'java*'
Note, selecting 'java2html' for glob 'java*'
Note, selecting 'java9-sdk-headless' for glob 'java*'
Note, selecting 'java-gcj-compat' for glob 'java*'
Note, selecting 'javascript-common' for glob 'java*'
Note, selecting 'java11-runtime' for glob 'java*'
Note, selecting 'javacc4' for glob 'java*'
Note, selecting 'java3ds-fileloader' for glob 'java*'
Note, selecting 'java10-runtime' for glob 'java*'
Note, selecting 'javahelp2-doc' for glob 'java*'
Note, selecting 'java-sdk-headless' for glob 'java*'
Note, selecting 'java-sdk' for glob 'java*'
Note, selecting 'java6-sdk-headless' for glob 'java*'
Note, selecting 'java11-sdk' for glob 'java*'
Note, selecting 'java10-sdk-headless' for glob 'java*'
Note, selecting 'javalex' for glob 'java*'
Note, selecting 'java-compiler' for glob 'java*'
Note, se

In [ ]:
#Initialize Spark Runtime Environment
!export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/jre
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import SQLContext

spark = SparkSession.builder \
   .master("local") \
   .appName("Wordcount & Friends Tutorial") \
   .config("spark.executor.memory", "2gb") \
   .getOrCreate()
   
sc = spark.sparkContext
sqlContext = SQLContext(sc)

sc.version

'3.0.1'

PART-1:  "WORDCOUNT" EXAMPLE USING MAPREDUCE

In [ ]:
# Fetch the text file for wordcount example
!wget "https://raw.githubusercontent.com/preethambonthula/AnalyticsHW7/main/WordCount"
!cat WordCount

--2020-11-05 22:11:42--  https://raw.githubusercontent.com/preethambonthula/AnalyticsHW7/main/WordCount
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1088 (1.1K) [text/plain]
Saving to: ‘WordCount.1’

WordCount.1         100%[===================>]   1.06K  --.-KB/s    in 0s      

2020-11-05 22:11:42 (59.8 MB/s) - ‘WordCount.1’ saved [1088/1088]

Machine learning (ML) is the study of computer algorithms that improve automatically through experience.
It is seen as a subset of artificial intelligence. 
Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so.
Machine learning algorithms are used in a wide variety of applications, such as email f

In [ ]:
#Simple example to read text file
rdd0 = sc.textFile("WordCount")
rdd0.take(20)

['Machine learning (ML) is the study of computer algorithms that improve automatically through experience.',
 'It is seen as a subset of artificial intelligence. ',
 'Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so.',
 'Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, ',
 'where it is difficult or infeasible to develop conventional algorithms to perform the needed tasks. ',
 'A subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers.',
 'But not all machine learning is statistical learning.',
 'The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.',
 'Data mining is a related field of study, focusing on exploratory data analysis through unsupervised learni

In [ ]:
word_counts = rdd0.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: (a + b)).map(lambda x:(x[1],x[0]))

word_counts.take(30)


[(3, 'Machine'),
 (6, 'learning'),
 (1, '(ML)'),
 (7, 'is'),
 (3, 'the'),
 (2, 'study'),
 (7, 'of'),
 (2, 'computer'),
 (4, 'algorithms'),
 (1, 'that'),
 (1, 'improve'),
 (1, 'automatically'),
 (2, 'through'),
 (1, 'experience.'),
 (1, 'It'),
 (1, 'seen'),
 (4, 'as'),
 (4, 'a'),
 (2, 'subset'),
 (1, 'artificial'),
 (1, 'intelligence.'),
 (1, 'build'),
 (1, 'model'),
 (1, 'based'),
 (3, 'on'),
 (1, 'sample'),
 (1, 'data,'),
 (1, 'known'),
 (1, '"training'),
 (1, 'data",')]

PART-2:  "COMMON FRIENDS" EXAMPLE USINg MAPREDUCE

In [ ]:
# Load data from github
!wget "https://raw.githubusercontent.com/preethambonthula/AnalyticsHW7/main/Friends"
rdd = sc.textFile("Friends")
!cat Friends

--2020-11-05 22:12:34--  https://raw.githubusercontent.com/preethambonthula/AnalyticsHW7/main/Friends
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 195 [text/plain]
Saving to: ‘Friends’

Friends             100%[===================>]     195  --.-KB/s    in 0s      

2020-11-05 22:12:34 (10.3 MB/s) - ‘Friends’ saved [195/195]

me Bharath
Varun me 
Varun Bharath
me Shiva 
Shiva Bharath
Varun Ravi
Ravi Shiva 
Shiva Varun
Adithya me 
Ravi Adithya 
Ravi Bharath
Kaushik Varun 
Bharath Kaushik 
Kaushik Adithya


In [ ]:
# Print the RDD content
rdd.take(20)

['me Bharath',
 'Varun me ',
 'Varun Bharath',
 'me Shiva ',
 'Shiva Bharath',
 'Varun Ravi',
 'Ravi Shiva ',
 'Shiva Varun',
 'Adithya me ',
 'Ravi Adithya ',
 'Ravi Bharath',
 'Kaushik Varun ',
 'Bharath Kaushik ',
 'Kaushik Adithya']

In [ ]:
rdd1=rdd.map(lambda x: x.split()).union(rdd.map(lambda x: x.split()[::-1]))
# Bring my friend list to local
lst = rdd1.filter(lambda x: x[0] == 'me').map(lambda x: x[1]).collect()
# Build the second pair RDD
rdd2 = rdd1.filter(lambda x: x[0] in lst).map(lambda x: x[1]). \
    filter(lambda x: x != 'me' and x not in lst). \
    map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b). \
    map(lambda x: (x[1], x[0])).sortByKey(ascending = False)
# Bring the result to local since the sample is small
for x, y in rdd2.collect():
   print ("The stranger {} has {} common friends with me".format(y, x))


The stranger Ravi has 4 common friends with me
The stranger Kaushik has 3 common friends with me
